# OpenCV Object Detection

- Casecade Classifier
- Casecade Classifier Training


In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


## Casecade Classifier
<img src="resource/lena_face.png" style="width:250px"></img><br>
- Object Detection OpenCV is using **Haar feature-based cascade classifiers.**
- Is an effective object detection method proposed by Paul Viola and Michael Jones in their paper, "Rapid Object Detection using a Boosted Cascade of Simple Features" in 2001.
- **Haar features** just like normal convolutional kernel, <br><br>
<img src="resource/haar_feature.png"></img><br><br>
- Each feature is a single value obtained by **subtracting** sum of pixels under the **white rectangle** from sum of pixels under the **black rectangle**. <br><br>
<img src="resource/face_haar_feature.png" style="width:600px"></img><br><br>
- `lena.jpg` convolving proses to detect face using haar feature, <br><br>
<img src="resource/convolving_haar_feature.gif" style="width:250px"></img>

- Class `cv2.CascadeClassifier()` digunakan untuk membaca classifier file (**.xml**)
- Pada class `cv2.CascadeClassifier()` terdapat method `.detectMultiscale()` untuk melakukan deteksi objek pada sebuah citra.

- Method `.detectMultiscale()` memiliki beberapa parameter input,
    - `scaleFactor` : Ukuran seberapa besar input image direduksi agar mampu dibaca oleh detector algorithm. Hal inilah yang memungkinkan algorima dapat mendeteksi wajah dalam beragam skala gambar (multi scale image).
    - `minNeighbors` : Ukuran minimum antara posisi face rectangle satu terhadap lainya. Hal ini berkaitan dengan method `.detectMultiscale()` yang akan melakukan sliding window terhadap image. Jika kita set ke 0, maka banyak false positive face rectangle terdeteksi. sehingga kita akan pilih nilai yang lebih tinggi. Namun jangan sampai memilih nilai yang terlalu besar, yang mengakibatkan true positive face rectangle menjadi tidak terdeteksi.
    - `flags` : Parameter yang sama pada method cvHaarDetectObjects. Ini tidak digunakan pada Cascade Classifier terbaru.
    - `minSize` : Ukuran object minimal. Ukuran yang lebih kecil tidak akan dimasukan kedalam detected object.
    - `maxSize` : Ukuran object maksimal. Ukuran yang lebih besar tidak akan dimasukan kedalam detected object.

- Face Detection (`haarcascade_frontalface_default.xml`)

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

img = cv2.imread("lena.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

cv2.imshow('img',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Eye Detection (`haarcascades/haarcascade_eye.xml`)

In [3]:
eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_eye.xml')

img = cv2.imread("eye.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

eye = eye_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in eye:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

cv2.imshow('img',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Face Detection (haarcascade_frontalface_default.xml)
    - Eye Detection (`haarcascades/haarcascade_eye.xml`)

In [4]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_smile.xml')

img = cv2.imread("lena.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (x,y,w,h) in eyes:
        cv2.rectangle(roi_color,(x,y),(x+w,y+h),(0,255,0),2)
        
    smile = smile_cascade.detectMultiScale(roi_gray, 1.5, 7)
    for (x,y,w,h) in smile:
        cv2.rectangle(roi_color,(x,y),(x+w,y+h),(0,0,255),2)

cv2.imshow('img',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Detect Face from Camera

In [6]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.imshow('Detect Face', frame)
        if cv2.waitKey(25) == ord('q'):
            break
    else :
        break

cv2.destroyAllWindows()
cap.release()

- Using custom box (`draw_ped()`)

In [2]:
def draw_ped(img, label, x0, y0, xt, yt, color=(255,127,0), text_color=(255,255,255)):

    (w, h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(img,
                  (x0, y0 + baseline),  
                  (max(xt, x0 + w), yt), 
                  color, 
                  2)
    cv2.rectangle(img,
                  (x0, y0 - h),  
                  (x0 + w, y0 + baseline), 
                  color, 
                  -1)  
    cv2.putText(img, 
                label, 
                (x0, y0),                   
                cv2.FONT_HERSHEY_SIMPLEX,     
                0.5,                          
                text_color,                
                1,
                cv2.LINE_AA) 
    return img

- Apply to Image

In [17]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

img = cv2.imread("family.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    img = draw_ped(img, "human face", x, y, x + w, y + h)

cv2.imshow('img',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        for (x, y, w, h) in faces:
            frame = draw_ped(frame, "human face", x, y, x + w, y + h)
        cv2.imshow('Detect Face', frame)
    else :
        break
    if cv2.waitKey(25) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()

- cars detection

In [ ]:
cars_cascade = cv2.CascadeClassifier('haarcascades/cars.xml')

img = cv2.imread("cars2.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cars = cars_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 2)
for (x,y,w,h) in cars:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
cv2.imshow('img',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Detect Cars from video (`haarcascades/cars.xml`)

video source : [link](https://www.pexels.com/video/cars-on-the-road-854745/)

In [20]:
car_cascade = cv2.CascadeClassifier('haarcascades/cars_v2.xml')

cap = cv2.VideoCapture("cars_video.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        cars = car_cascade.detectMultiScale(gray, 1.1, 5)
        for (x, y, w, h) in cars:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
            
        cv2.imshow('Detect Cars', frame)

        if cv2.waitKey(1) == ord('q'):
            break
    else :
        break
cv2.destroyAllWindows()
cap.release()

- Full Body detection

In [38]:
full_body_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_fullbody.xml')

cap = cv2.VideoCapture("pedestrians.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        
        frame = cv2.resize(frame, (0,0), fx=0.4, fy=0.4)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        full_bodies = full_body_cascade.detectMultiScale(gray, 1.1, 5)
        for (x, y, w, h) in full_bodies:
            frame = draw_ped(frame, "full body", x, y, x + w, y + h)
            
        cv2.imshow('Detect Cars', frame)

        if cv2.waitKey(1) == ord('q'):
            break
    else :
        break
cv2.destroyAllWindows()
cap.release()

___
## Cascade Classifier Training
- For training a boosted cascade of weak classifiers we need a set of **positive images** (containing actual objects you want to detect) and a set of **negative images** (containing everything you do not want to detect). 
- The set of **negative images** must be prepared manually, whereas set of positive samples is created using the `opencv_createsamples` application.

#### Negative Samples
- Negative samples are taken from arbitrary images, not containing objects you want to detect. 
- These negative images, from which the samples are generated, should be listed in a special negative image file containing one image path per line (can be absolute or relative). 
- Note that negative samples and sample images are also called background samples or background images, and are used interchangeably in this document.
- Directory Structure :

> /img <br>
>  &emsp;img1.jpg<br>
>  &emsp;img2.jpg<br>
> bg.txt

- File bg.txt:
> img/img1.jpg<br>
> img/img2.jpg

#### Positive Samples
- Positive samples are created by the `opencv_createsamples` application. 
- They are used by the boosting process to define what the model should actually look for when trying to find your objects of interest. 
- The application supports two ways of generating a positive sample dataset.

## Haar Training GUI Application

Created by : [Amin Ahmadi - cascade-trainer-gui](https://amin-ahmadi.com/cascade-trainer-gui/)

![](resource/cascade-trainer-gui-01-train-input-768x540.png)

### Casecade Classifier Training 
- n sample : 24000
- p sample : 5000
- n stage : 7
- Feature : LBP
- HR : 0.995
- FA : 0.2
- WxH : 32x32

![](resource/cascade-trainer-guis-02.png)